In [4]:
import pandas as pd
import numpy as np
import os
import sys
import scanpy as sc
import anndata as ad
import scipy
import gget
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

In [8]:
# load predictions
fpath = "/nfs/turbo/umms-indikar/shared/projects/MC3R/hypomap/merged_predictions.csv"
df = pd.read_csv(fpath)
print(f"{df.shape=}")

for c in df.columns:
    if not c == 'index':
        df[f'{c}_clean'] = df[c].apply(lambda x: x.split(":")[1]).astype(str)


# load clustering results
fpath = "/nfs/turbo/umms-indikar/shared/projects/MC3R/results/embedding.csv"
cdf = pd.read_csv(fpath)
print(f"{cdf.shape=}")

# merge 'em'
df = pd.merge(df, cdf, 
              how='left',
              left_on='index',
              right_on='index')


# clean-up the cell type
df = df.set_index('index')

print(f"{df.shape=}")
df.head()

df.shape=(43312, 5)
cdf.shape=(43312, 3)
df.shape=(43312, 10)


,reference_embedding_C7_named,reference_embedding_C25_named,C25_named,C7_named,reference_embedding_C7_named_clean,reference_embedding_C25_named_clean,C25_named_clean,C7_named_clean,UMAP1,UMAP2
index,,,,,,,,,,
AAACCCACATGGCTGC-1_fasted_m,C7-2: GABA,C25-11: GABA-1,C25-10: GABA-2,C7-2: GABA,GABA,GABA-1,GABA-2,GABA,-6.741960,14.083050
AAACCCAGTATATGGA-1_fasted_m,C7-2: GABA,C25-13: GABA-4,C25-1: GLU-1,C7-2: GABA,GABA,GABA-4,GLU-1,GABA,2.378481,2.956433
AAACCCAGTCAACACT-1_fasted_m,C7-5: Immune,C25-21: Immune,C25-21: Immune,C7-5: Immune,Immune,Immune,Immune,Immune,1.525248,-6.143299
AAACCCATCCCGTGAG-1_fasted_m,C7-4: Oligo+Precursor,C25-19: Oligodendrocytes,C25-19: Oligodendrocytes,C7-4: Oligo+Precursor,Oligo+Precursor,Oligodendrocytes,Oligodendrocytes,Oligo+Precursor,15.965842,14.408034
AAACCCATCGAACTCA-1_fasted_m,C7-1: GLU,C25-2: GLU-2,C25-2: GLU-2,C7-1: GLU,GLU,GLU-2,GLU-2,GLU,-2.045243,7.478702


In [23]:
bdf = df.copy()

keep = [
    'GABA', 
    # 'Immune',
    'OligoPrecursor', 
    'GLU', 
    'AstroEpendymal',
    # 'Vascular', 
    # 'ParsTuber'
]

bdf = bdf.reset_index(drop=False)

bdf = bdf[['index', 'reference_embedding_C7_named_clean']]
bdf.columns = ['cellId','cellType']
bdf['barcode'] = bdf['cellId'].apply(lambda x: x.split("-")[0])
bdf['diet'] = bdf['cellId'].apply(lambda x: x.split("1_")[1].split("_")[0])
bdf['sex'] = bdf['cellId'].apply(lambda x: x.split("1_")[1].split("_")[1])
bdf['cellType'] = bdf['cellType'].apply(lambda x: x.strip().replace("+", "").replace("-", ""))
bdf['key'] = bdf['diet'] + bdf['sex']

bdf = bdf[bdf['cellType'].isin(keep)]

outpath = "/nfs/turbo/umms-indikar/shared/projects/MC3R/isoforms/barcodes.csv"
bdf.to_csv(outpath, index=False)
bdf.head()

,cellId,cellType,barcode,diet,sex,key
0,AAACCCACATGGCTGC-1_fasted_m,GABA,AAACCCACATGGCTGC,fasted,m,fastedm
1,AAACCCAGTATATGGA-1_fasted_m,GABA,AAACCCAGTATATGGA,fasted,m,fastedm
3,AAACCCATCCCGTGAG-1_fasted_m,OligoPrecursor,AAACCCATCCCGTGAG,fasted,m,fastedm
4,AAACCCATCGAACTCA-1_fasted_m,GLU,AAACCCATCGAACTCA,fasted,m,fastedm
5,AAACCCATCTTGCGCT-1_fasted_m,GLU,AAACCCATCTTGCGCT,fasted,m,fastedm


In [ ]:
keep = [
    'GABA', 
    'OligoPrecursor', 
    'GLU',
    'AstroEpendymal',
]

In [21]:
bdf['cellType'].unique()

array(['GABA', 'Immune', 'OligoPrecursor', 'GLU', 'AstroEpendymal',
       'Vascular', 'ParsTuber'], dtype=object)

In [ ]:
subdir_path = "filtered_feature_bc_matrix"

data_names = {
    "Sample_7392-GD-4-GEX_ACAATGTGAA-TAACGGTACG" : "fasted_f",
    "Sample_7392-GD-1-GEX_TTATTCGAGG-AGCAGGACAG" : "fed_m",
    "Sample_7392-GD-3-GEX_TGTAGTCATT-TACGATCAAG" : "fed_f",
    "Sample_7392-GD-2-GEX_AAGATTGGAT-AAATCCCGCT" : "fasted_m",
}
